In [2]:
from datetime import datetime, timedelta
from pytrends.request import TrendReq
import pandas as pd
import os


In [5]:
maxstep = 269
overlap = 40
step    = maxstep - overlap + 1
kw_list = ['TSLA']
start_date = datetime(2018, 12, 9).date()

In [6]:
pytrend = TrendReq()

In [7]:
today = datetime.today().date()
old_date = today

In [8]:
new_date = today - timedelta(days=step)

In [9]:
timeframe = new_date.strftime('%Y-%m-%d')+' '+old_date.strftime('%Y-%m-%d')
pytrend.build_payload(kw_list=kw_list, timeframe = timeframe)
interest_over_time_df = pytrend.interest_over_time()


In [10]:
while new_date>start_date:
    old_date = new_date + timedelta(days=overlap-1)
    new_date = new_date - timedelta(days=step)
    if new_date < start_date:
        new_date = start_date
    timeframe = new_date.strftime('%Y-%m-%d')+' '+old_date.strftime('%Y-%m-%d')
    print(timeframe)

    pytrend.build_payload(kw_list=kw_list, timeframe = timeframe)
    temp_df = pytrend.interest_over_time()
    if (temp_df.empty):
        raise ValueError('Google sent back an empty dataframe. Possibly there were no searches at all during the this period! Set start_date to a later date.')
    for kw in kw_list:
        beg = new_date
        end = old_date - timedelta(days=1)
        for t in range(1,overlap+1):
            if temp_df[kw].iloc[-t] != 0:
                scaling = interest_over_time_df[kw].iloc[t-1]/temp_df[kw].iloc[-t]
                break
            elif t == overlap:
                print('Did not find non-zero overlap, set scaling to zero! Increase Overlap!')
                scaling = 0
        temp_df.loc[beg:end,kw]=temp_df.loc[beg:end,kw]*scaling
    interest_over_time_df = pd.concat([temp_df[:-overlap],interest_over_time_df])


In [11]:
interest_over_time_df.head()

,TSLA,isPartial
date,,
2018-08-15,30,False
2018-08-16,26,False
2018-08-17,44,False
2018-08-18,15,False
2018-08-19,10,False


In [15]:
interest_over_time_df.drop(columns='isPartial')


,TSLA
date,
2018-08-15,30
2018-08-16,26
2018-08-17,44
2018-08-18,15
2018-08-19,10
2018-08-20,48
2018-08-21,35
2018-08-22,27
2018-08-23,20
